In [ ]:
# If PEPit is not installed yet, you can run this cell.
!pip install pepit

# Code description

In this notebook, we numerically verify Lemma C.2 from 

> "Convergence of Proximal Point and Extragradient-Based Methods Beyond Monotonicity: the Case of Negative Comonotonicity"

## Problem

We consider the following problem
       $$ \text{find } x^\ast \text{ such that }F(x^\ast) = 0$$
where $F:\mathbb{R}^d \to \mathbb{R}^d$ is a maximal $\rho$-negatively comonotone and $L$-Lipschitz operator.

## Algorithm

We consider the Extragradient method (EG) with stepsize $\gamma_1=\gamma_2=\gamma$ policy: for all $k \geq 0$
$$
\tilde{x}^k = x^k - \gamma F(x^{k}),
$$
$$
x^{k+1} = x^k - \gamma F(\tilde{x}^k)
$$

## The goal

Lemma C.2 states that for all $k \geq 0$ and for any $\gamma \geq 0$, $L \geq 0$ and $\rho \geq 0$
$$
\|F(x^{k+1})\|^2 \leq \|F(x^k)\|^2- \left(\frac{1}{2} - 2L^2\gamma^2\right)\|F(\tilde{x}^k) - F(x^k)\|^2 - \left(\frac{1}{2} - \frac{\rho}{\gamma}\right)\|F(\tilde{x}^k) - F(x^{k+1})\|^2 - \left(\frac{1}{2} - \frac{2\rho}{\gamma}\right)\|F(x^k) - F(x^{k+1})\|^2.
$$

Below we verify this inequality numerically.

In [1]:
# Run this code before executing the cell below
from math import sqrt
import numpy as np
from PEPit import PEP

from PEPit.function import Function


class LipschitzNegativeComonotoneOperator(Function):
    
    def __init__(self,
                 rho,
                 L=1.,
                 is_leaf=True,
                 decomposition_dict=None,
                 reuse_gradient=True):
        
        super().__init__(is_leaf=is_leaf,
                         decomposition_dict=decomposition_dict,
                         reuse_gradient=True)
        # Store L and rho
        self.rho = rho
        self.L = L

        if self.L == np.inf:
            print("\033[96m(PEPit) The class of Lipschitz negative comonotone operators is necessarily continuous.\n"
                  "To instantiate an operator, please avoid using the class LipschitzNegativeComonotoneOperator with\n"
                  " L == np.inf. Instead, please use the class NegativeComonotoneOperator (which accounts for the fact\n"
                  "that the image of the operator at certain points might not be a singleton).\033[0m")

    def add_class_constraints(self):
        """
        Formulates the list of necessary conditions for interpolation of self (Lipschitz negative comonotone and
        maximally negative comonotone operator), see, e.g., discussions in [1, Section 2].
        """

        for point_i in self.list_of_points:

            xi, gi, fi = point_i

            for point_j in self.list_of_points:

                xj, gj, fj = point_j

                if (xi != xj) | (gi != gj):
                    # Interpolation conditions of negative comonotone operator class
                    self.add_constraint((gi - gj) * (xi - xj) + self.rho * (gi - gj)**2 >= 0)
                    # Interpolation conditions of Lipschitz operator class
                    self.add_constraint((gi - gj)**2 - self.L**2 * (xi - xj)**2 <= 0)

In [2]:
##########################################################
# parameters: MODIFY HERE!

# pick the parameters for which you want to verify
# the inequality (numerically)
L = 1;
gamma = 100;
rho = 10;

# verbose & verification tolerance options
verbose = 0;
tolerance = 1e-5;
##########################################################



# (0) Initialize an empty PEP
problem = PEP()

# (1) Set up the problem class
L  =  L; rho = rho; # F is 1-Lipschitz and \rho-negative comonotone
F  = problem.declare_function(LipschitzNegativeComonotoneOperator, L=L, rho=rho)
xs = F.stationary_point();  # x^* is a solution

# (2) Set up the starting points
x0 = problem.set_initial_point() # this is x^0

# (3) Run the algorithm
tx0     = x0 - gamma * F.gradient(x0); 
x1      = x0 - gamma * F.gradient(tx0);

# (3) define the expressions (recall that our objective is to verify that
#     psi1 - psi0 - residual<=0 for all F and sequence generated by the
#     extragradient method.

psi0 = F.gradient(x0)**2;
psi1 = F.gradient(x1)**2;
term1 = - (0.5 - 2 * (L**2) * (gamma**2)) * (F.gradient(tx0) - F.gradient(x0))**2
term2 = - (0.5 - rho / gamma) * (F.gradient(tx0) - F.gradient(x1))**2
term3 = - (0.5 - 2 * rho / gamma) * (F.gradient(x0) - F.gradient(x1))**2
residual =  term1 + term2 + term3;

# (4) Set up the performance measure: (PEPit will compute its maximum value numerically)
expression_to_verify = psi1 - psi0 - residual;
problem.set_performance_metric(expression_to_verify);

# (5) Solve the PEP
worstcase_value = problem.solve(verbose=verbose)


# (6) is the potential verified? Success if MAX(expression_to_verify) <= numerical_tolerance for the 
#     choice of the parameters above.
print('Did PEPit verify the potential (within prescribed numerical precision)? {:}  \t'.format(worstcase_value<tolerance))
worstcase_value # this should be close to zero

Did PEPit verify the potential (within prescribed numerical precision)? True  	


-1.3201570947529717e-09